# ROI Filter Comparison

This notebook detects Regions of Interest (ROI) using **six distinct filtering approaches** side-by-side.

## Filters:
1.  **Baseline**: Gradient Amplitude + Anisotropy. (From `detect_roi.py`)
2.  **Harris Corner Density**: High density of corners/intersections.
3.  **Local Variance**: Statistical variance of pixel intensity (Texture energy).
4.  **Edge Density**: Density of Canny edges.
5.  **Finder Pattern Match**: Template matching for the "Bullseye" corners (1:1:3:1:1 ratio).
6.  **Gabor Cross-Energy**: Response to *both* horizontal and vertical high-frequencies (cross-hatch).

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed
from pathlib import Path

%matplotlib inline

## 1. Helper Functions (Normalization & Box Extraction)

In [3]:
def normalize01(x: np.ndarray) -> np.ndarray:
    if x is None: return None
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx - mn < 1e-6:
        return np.zeros_like(x, dtype=np.float32)
    return (x - mn) / (mx - mn)

def extract_and_draw_boxes(img_rgb, score_map, patch_size=128, top_k=50, color=(0, 255, 0)):
    """Extract NMS boxes from score map and draw them on a copy of the image."""
    h, w = score_map.shape[:2]
    s_map = score_map.copy()
    half = patch_size // 2
    
    out_img = img_rgb.copy()
    count = 0
    
    for _ in range(top_k):
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(s_map)
        if max_val < 0.1: break
        
        cx, cy = max_loc
        x1 = max(0, cx - half)
        y1 = max(0, cy - half)
        x2 = min(w, x1 + patch_size)
        y2 = min(h, y1 + patch_size)
        
        # Draw
        cv2.rectangle(out_img, (x1, y1), (x2, y2), color, 2)
        
        # Strict NMS: Zero out 2*patch_size to prevent ANY overlap
        pad = int(patch_size * 2.0)
        nx1 = max(0, cx - pad//2)
        ny1 = max(0, cy - pad//2)
        nx2 = min(w, cx + pad//2)
        ny2 = min(h, cy + pad//2)
        s_map[ny1:ny2, nx1:nx2] = 0
        count += 1
        
    return out_img, count

## 2. Filter Implementations

In [4]:
# --- 1. Baseline (Gradient + Anisotropy) ---
def filter_baseline(gray, sigma=2.0, w_grad=0.4, w_aniso=0.6, k_blur=5):
    g = cv2.GaussianBlur(gray, (k_blur, k_blur), 0)
    dx = cv2.Sobel(g, cv2.CV_32F, 1, 0, ksize=3)
    dy = cv2.Sobel(g, cv2.CV_32F, 0, 1, ksize=3)
    mag = cv2.magnitude(dx, dy)
    
    jxx = dx * dx
    jyy = dy * dy
    jxy = dx * dy
    
    k = int(max(3, round(sigma * 6))) | 1
    jxx = cv2.GaussianBlur(jxx, (k, k), sigma)
    jyy = cv2.GaussianBlur(jyy, (k, k), sigma)
    jxy = cv2.GaussianBlur(jxy, (k, k), sigma)
    
    tr = jxx + jyy
    det = jxx * jyy - jxy * jxy
    disc = np.maximum(tr * tr - 4.0 * det, 0.0)
    root = np.sqrt(disc)
    l1 = 0.5 * (tr + root)
    l2 = 0.5 * (tr - root)
    aniso = (l1 - l2) / (l1 + l2 + 1e-6)
    
    raw = w_grad * normalize01(mag) + w_aniso * normalize01(aniso)
    return normalize01(raw)

# --- 2. Harris Corner Density ---
def filter_harris(gray, block_size=2, k=0.04, density_radius=15, thresh_rel=0.01):
    harris = cv2.cornerHarris(gray, block_size, 3, k)
    thresh = thresh_rel * harris.max()
    corners_binary = (harris > thresh).astype(np.float32)
    
    d_k = density_radius * 2 + 1
    density = cv2.blur(corners_binary, (d_k, d_k))
    return normalize01(density)

# --- 3. Local Variance (Texture Energy) ---
def filter_variance(gray, window_size=21):
    if window_size % 2 == 0: window_size += 1
    img_f = gray.astype(np.float32) / 255.0
    mu = cv2.blur(img_f, (window_size, window_size))
    mu2 = cv2.blur(img_f * img_f, (window_size, window_size))
    variance = mu2 - mu * mu
    variance = np.maximum(variance, 0)
    return normalize01(np.sqrt(variance))

# --- 4. Edge Density ---
def filter_edge_density(gray, thresh1=50, thresh2=150, radius=15):
    edges = cv2.Canny(gray, thresh1, thresh2)
    edges_f = (edges > 0).astype(np.float32)
    d_k = radius * 2 + 1
    density = cv2.blur(edges_f, (d_k, d_k))
    return normalize01(density)

# --- 5. Finder Pattern Match (Bullseye) ---
def filter_finder_pattern(gray, module_size_px=3):
    """Match the 1:1:3:1:1 concentric square pattern."""
    ms = max(1, module_size_px)
    template_size = 7 * ms
    
    tpl = np.zeros((template_size, template_size), dtype=np.float32)
    c = template_size // 2
    
    # Outer Black Ring
    tpl[:, :] = -1.0 
    # Outer White Ring (Width 1 module)
    w1_s = ms
    w1_e = template_size - ms
    tpl[w1_s:w1_e, w1_s:w1_e] = 1.0
    # Inner Black Box (3x3 modules)
    b_s = 2 * ms
    b_e = template_size - 2 * ms
    tpl[b_s:b_e, b_s:b_e] = -1.0
    
    # Normalize template mean=0
    tpl = tpl - tpl.mean()
    
    # FIX: Convert gray to float32 to match template type for matchTemplate
    gray_f = gray.astype(np.float32)
    
    # Match
    match = cv2.matchTemplate(gray_f, tpl, cv2.TM_CCOEFF_NORMED)
    match = np.maximum(match, 0)
    
    # Resize back
    match = cv2.resize(match, (gray.shape[1], gray.shape[0]))
    
    return normalize01(match)

# --- 6. Gabor Cross-Energy ---
def filter_gabor_cross(gray, wavelength=5.0, sigma=3.0, k_size=15):
    """Geometric mean of horizontal and vertical Gabor energy."""
    kern_h = cv2.getGaborKernel((k_size, k_size), sigma, 0, wavelength, 1.0, 0, ktype=cv2.CV_32F)
    kern_v = cv2.getGaborKernel((k_size, k_size), sigma, np.pi/2, wavelength, 1.0, 0, ktype=cv2.CV_32F)
    
    gray_f = gray.astype(np.float32)
    filt_h = cv2.filter2D(gray_f, cv2.CV_32F, kern_h)
    filt_h = cv2.filter2D(gray_f, cv2.CV_32F, kern_v)
    
    # Oops, small bug previously: filter2D with same kern_v variable name in second call?
    # No, I used kern_v correctly but variable assignment was filt_h.
    # Correcting implementation:
    filt_h = cv2.filter2D(gray_f, cv2.CV_32F, kern_h)
    filt_v = cv2.filter2D(gray_f, cv2.CV_32F, kern_v)
    
    energy_h = filt_h * filt_h
    energy_v = filt_v * filt_v
    
    cross = np.sqrt(energy_h * energy_v)
    return normalize01(cross)

## 3. Interactive Comparison

In [6]:
# Load Images
data_dir = Path("../data/raw")
files = sorted(list(data_dir.glob("*.png")) + list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.jpeg")))
options = [(p.name, str(p)) for p in files]
if not options: options = [("No images", None)]

def show_comparison(
    image_path,
    # Base
    b_sigma, b_w_grad, b_w_aniso,
    # Harris
    h_bsize, h_thresh, h_rad,
    # Variance
    v_wsize,
    # Edges
    e_th1, e_rad,
    # Finder
    f_mod_size,
    # Gabor
    g_wave, g_sigma,
    # Global
    patch_size, top_k
):
    if not image_path: return
    
    img = cv2.imread(image_path)
    if img is None: return
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # --- Compute All ---
    map_base = filter_baseline(gray, sigma=b_sigma, w_grad=b_w_grad, w_aniso=b_w_aniso)
    map_harris = filter_harris(gray, block_size=h_bsize, thresh_rel=h_thresh, density_radius=h_rad)
    map_var = filter_variance(gray, window_size=v_wsize)
    map_edges = filter_edge_density(gray, thresh1=e_th1, thresh2=e_th1*3, radius=e_rad)
    map_finder = filter_finder_pattern(gray, module_size_px=f_mod_size)
    map_gabor = filter_gabor_cross(gray, wavelength=g_wave, sigma=g_sigma)
    
    maps = [
        ("1. Base (Grad+Aniso)", map_base),
        ("2. Harris Density", map_harris),
        ("3. Local Variance", map_var),
        ("4. Edge Density", map_edges),
        ("5. Finder Pattern", map_finder),
        ("6. Gabor Cross", map_gabor)
    ]
    
    # --- Plot 6 Rows ---
    fig, axes = plt.subplots(6, 2, figsize=(14, 30))
    
    for i, (name, smap) in enumerate(maps):
        # Left: Score Map
        ax_map = axes[i, 0]
        im = ax_map.imshow(smap, cmap='jet')
        ax_map.set_title(f"{name} - Heatmap")
        ax_map.axis('off')
        plt.colorbar(im, ax=ax_map, fraction=0.046, pad=0.04)
        
        # Right: Detection Boxes
        ax_res = axes[i, 1]
        res_img, count = extract_and_draw_boxes(img_rgb, smap, patch_size=patch_size, top_k=top_k)
        ax_res.imshow(res_img)
        ax_res.set_title(f"{name} - {count} Boxes")
        ax_res.axis('off')
    
    plt.tight_layout()
    plt.show()

interact(
    show_comparison, 
    image_path=widgets.Dropdown(options=options, description="Image:"),
    
    b_sigma=widgets.FloatSlider(value=2.0, min=0.5, max=5.0, description='Base Sigma'),
    b_w_grad=widgets.FloatSlider(value=0.4, min=0.0, max=2.0, description='Base W_Grad'),
    b_w_aniso=widgets.FloatSlider(value=0.6, min=0.0, max=2.0, description='Base W_Aniso'),
    
    h_bsize=widgets.IntSlider(value=2, min=2, max=7, description='Harris Block'),
    h_thresh=widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=-1, description='Harris Thresh'),
    h_rad=widgets.IntSlider(value=15, min=3, max=50, description='Harris Radius'),
    
    v_wsize=widgets.IntSlider(value=21, min=3, max=101, step=2, description='Var Window'),
    
    e_th1=widgets.IntSlider(value=50, min=10, max=200, description='Canny Thresh'),
    e_rad=widgets.IntSlider(value=15, min=3, max=50, description='Edge Radius'),
    
    f_mod_size=widgets.IntSlider(value=3, min=1, max=10, description='Finder Module Px'),
    
    g_wave=widgets.FloatSlider(value=5.0, min=2.0, max=20.0, description='Gabor Wavelength'),
    g_sigma=widgets.FloatSlider(value=3.0, min=1.0, max=10.0, description='Gabor Sigma'),
    
    patch_size=widgets.IntSlider(value=128, min=32, max=256, step=32, description='Patch Size'),
    top_k=widgets.IntSlider(value=50, min=1, max=100, description='Max Boxes'),
);

interactive(children=(Dropdown(description='Image:', options=(('001.png', '../data/raw/001.png'), ('002.png', …